# Recommender Network for Movies
by Alexander Köhn

## Information for the tester
In the following my whole Software contribution is shown via a Jupyter notebook. My Final Report is included during the Notebook at the relevant parts of my code.
The summary of the tutorials are in an extra file in my repository.

## Credits
I got information for this project from the Tutorial (https://keras.io/examples/structured_data/collaborative_filtering_movielens/) and (https://www.tensorflow.org/recommenders/examples/basic_retrieval)

## Project Goal and underlying topic
The Project Goal is to implement a recommender system for love pairs, which want to watch a movie together during the winter holidays. The motivation is to reduce the time couples spend searching for the right movie.
Common Platforms use recommender systems with the data of the signed account. These System are not achieving the best results, if the account is most often used alone by one person and just used sometimes to watch movies together.
Implementing a recommender system with the input of two persons, which want to watch a movie together, and a output of movies, which are good for both of them, is the Goal.

Additionally we want to have some more features.
1. We can decide which users interest is more important for the network.
2. We can decide if the network should recommend a movie they will likely watch (Retrieval) or they will likely like (Ranking)

## Inputs
First we need some Packages for building the network and analysing the data.

In [108]:
%pip install -q tensorflow-recommenders
%pip install -q --upgrade tensorflow-datasets
%pip install -q scann

Note: you may need to restart the kernel to use updated packages.


You should consider upgrading via the 'c:\users\koehn\pycharmprojects\pythonproject\venv\scripts\python.exe -m pip install --upgrade pip' command.


Note: you may need to restart the kernel to use updated packages.


You should consider upgrading via the 'c:\users\koehn\pycharmprojects\pythonproject\venv\scripts\python.exe -m pip install --upgrade pip' command.


Note: you may need to restart the kernel to use updated packages.


ERROR: Could not find a version that satisfies the requirement scann (from versions: none)
ERROR: No matching distribution found for scann
You should consider upgrading via the 'c:\users\koehn\pycharmprojects\pythonproject\venv\scripts\python.exe -m pip install --upgrade pip' command.


In [109]:
import os
import pprint
import tempfile

from typing import Dict, Text

import numpy as np
import tensorflow as tf
import tensorflow_datasets as tfds
import tensorflow_recommenders as tfrs

## Data
We are using the Movielens 100K Dataset, which features a set with 100,000 ratings (1-5) from 943 users on 1682 movies. Tensorflow already provides a method to download the Dataset to the hard disk. After the first download the dataset from the disk will be used.

In [110]:
# Ratings data.
ratings = tfds.load("movielens/100k-ratings", split="train")
# Features of all the available movies.
movies = tfds.load("movielens/100k-movies", split="train")

As you can see, each rating consists of a movie with specific genres and a user with demographic information.

In [111]:
for x in ratings.take(1).as_numpy_iterator():
  pprint.pprint(x)

{'bucketized_user_age': 45.0,
 'movie_genres': array([7], dtype=int64),
 'movie_id': b'357',
 'movie_title': b"One Flew Over the Cuckoo's Nest (1975)",
 'raw_user_age': 46.0,
 'timestamp': 879024327,
 'user_gender': True,
 'user_id': b'138',
 'user_occupation_label': 4,
 'user_occupation_text': b'doctor',
 'user_rating': 4.0,
 'user_zip_code': b'53211'}


In [112]:
for x in movies.take(1).as_numpy_iterator():
  pprint.pprint(x)

{'movie_genres': array([4], dtype=int64),
 'movie_id': b'1681',
 'movie_title': b'You So Crazy (1994)'}


## Analaysis of the data
We can gain a better intuition for the dataset if we gather information about statistical information of the dataset.


## Prepocessing of the data


To simplify our model we will only use the user_id, movie_title and the user_rating from the Dataset.

In [113]:
ratings = ratings.map(lambda x: {
    "movie_title": x["movie_title"],
    "user_id": x["user_id"],
    "user_rating": x["user_rating"],
})
movies = movies.map(lambda x: x["movie_title"])

We are shuffling the data of the rating to randomize the learning and to avoid a bias in the train and test data.

In [114]:
tf.random.set_seed(37)
shuffled = ratings.shuffle(100_000, seed=37, reshuffle_each_iteration=False)

We are dividing the dataset in 80 % train data and 20% test data.

In [115]:
train = shuffled.take(80_000)
test = shuffled.skip(80_000).take(20_000)

Now we want to get two lists. The first list unique_movie_titles consists of all the movie titles, but every title is only saved once in the list. The second list unique_user_ids consists of all the users, in the same was as the first.

In [116]:
movie_titles = movies.batch(1_000)
user_ids = ratings.batch(1_000_000).map(lambda y: y["user_id"])

unique_movie_titles = np.unique(np.concatenate(list(movie_titles)))
unique_user_ids = np.unique(np.concatenate(list(user_ids)))

print("First 10 Movie titles: ")
print(unique_movie_titles[:10])

print("\nThe total count of users: ", len(unique_user_ids))
print("The total count of movies: ", len(unique_movie_titles))

First 10 Movie titles: 
[b"'Til There Was You (1997)" b'1-900 (1994)' b'101 Dalmatians (1996)'
 b'12 Angry Men (1957)' b'187 (1997)' b'2 Days in the Valley (1996)'
 b'20,000 Leagues Under the Sea (1954)' b'2001: A Space Odyssey (1968)'
 b'3 Ninjas: High Noon At Mega Mountain (1998)' b'39 Steps, The (1935)']

The total count of users:  943
The total count of movies:  1664


## Architecture of our model
The Goal is to give the

In [117]:
class MovielensModel(tfrs.models.Model):

  def __init__(self, rating_weight: float, retrieval_weight: float) -> None:
    # We take the loss weights in the constructor: this allows us to instantiate
    # several model objects with different loss weights.

    super().__init__()

    embedding_dimension = 32

    # User and movie models.
    self.movie_model: tf.keras.layers.Layer = tf.keras.Sequential([
      tf.keras.layers.StringLookup(
        vocabulary=unique_movie_titles, mask_token=None),
      tf.keras.layers.Embedding(len(unique_movie_titles) + 1, embedding_dimension)
    ])
    self.user_model: tf.keras.layers.Layer = tf.keras.Sequential([
      tf.keras.layers.StringLookup(
        vocabulary=unique_user_ids, mask_token=None),
      tf.keras.layers.Embedding(len(unique_user_ids) + 1, embedding_dimension)
    ])

    # A small model to take in user and movie embeddings and predict ratings.
    # We can make this as complicated as we want as long as we output a scalar
    # as our prediction.
    self.rating_model = tf.keras.Sequential([
        tf.keras.layers.Dense(256, activation="relu"),
        tf.keras.layers.Dense(128, activation="relu"),
        tf.keras.layers.Dense(1),
    ])

    # The tasks.
    self.rating_task: tf.keras.layers.Layer = tfrs.tasks.Ranking(
        loss=tf.keras.losses.MeanSquaredError(),
        metrics=[tf.keras.metrics.RootMeanSquaredError()],
    )
    self.retrieval_task: tf.keras.layers.Layer = tfrs.tasks.Retrieval(
        metrics=tfrs.metrics.FactorizedTopK(
            candidates=movies.batch(128).map(self.movie_model)
        )
    )

    # The loss weights.
    self.rating_weight = rating_weight
    self.retrieval_weight = retrieval_weight

  def call(self, features: Dict[Text, tf.Tensor]) -> tf.Tensor:
    # We pick out the user features and pass them into the user model.
    user_embeddings = self.user_model(features["user_id"])
    # And pick out the movie features and pass them into the movie model.
    movie_embeddings = self.movie_model(features["movie_title"])

    return (user_embeddings, movie_embeddings,
        # We apply the multi-layered rating model to a concatentation of
        # user and movie embeddings.
        self.rating_model(
            tf.concat([user_embeddings, movie_embeddings], axis=1)
        ),
    )

  def compute_loss(self, features: Dict[Text, tf.Tensor], training=False) -> tf.Tensor:
    ratings = features.pop("user_rating")
    user_embeddings, movie_embeddings, rating_predictions = self(features)

    # We compute the loss for each task.
    rating_loss = self.rating_task(
        labels=ratings,
        predictions=rating_predictions,
    )
    retrieval_loss = self.retrieval_task(user_embeddings, movie_embeddings)

    # And combine them using the loss weights.
    return (self.rating_weight * rating_loss
            + self.retrieval_weight * retrieval_loss)


## Building our model

In [118]:
model = MovielensModel(rating_weight=1.0, retrieval_weight=0.0)
model.compile(optimizer=tf.keras.optimizers.Adagrad(0.1))

In [119]:
cached_train = train.shuffle(100_000).batch(8192).cache()
cached_test = test.batch(4096).cache()

In [120]:
model.fit(cached_train, epochs=3)
metrics = model.evaluate(cached_test, return_dict=True)

print(f"Retrieval top-100 accuracy: {metrics['factorized_top_k/top_100_categorical_accuracy']:.3f}.")
print(f"Ranking RMSE: {metrics['root_mean_squared_error']:.3f}.")

Epoch 1/3
10/10 [==============================] - 10s 856ms/step - root_mean_squared_error: 2.1771 - factorized_top_k/top_1_categorical_accuracy: 4.0000e-04 - factorized_top_k/top_5_categorical_accuracy: 0.0027 - factorized_top_k/top_10_categorical_accuracy: 0.0052 - factorized_top_k/top_50_categorical_accuracy: 0.0276 - factorized_top_k/top_100_categorical_accuracy: 0.0572 - loss: 4.3510 - regularization_loss: 0.0000e+00 - total_loss: 4.3510
Epoch 2/3
10/10 [==============================] - 12s 1s/step - root_mean_squared_error: 1.1229 - factorized_top_k/top_1_categorical_accuracy: 3.8750e-04 - factorized_top_k/top_5_categorical_accuracy: 0.0028 - factorized_top_k/top_10_categorical_accuracy: 0.0053 - factorized_top_k/top_50_categorical_accuracy: 0.0275 - factorized_top_k/top_100_categorical_accuracy: 0.0572 - loss: 1.2618 - regularization_loss: 0.0000e+00 - total_loss: 1.2618
Epoch 3/3
5/5 [==============================] - 3s 327ms/step - root_mean_squared_error: 1.1126 - factoriz